# Revisão:

## Middleware

Middleware é uma função que intercepta e processa as requisições e respostas no ciclo de vida de uma aplicação. Ele permite adicionar funcionalidades (como autenticação, logs, ou limitação de requisições) sem modificar diretamente as rotas.

### Como funciona o fluxo no FastAPI?

1. Cliente faz a requisição.
2. Middleware processa a requisição antes de chegar à rota.
3. Após o processamento, o middleware envia a resposta de volta.

### Por que usar Middlewares?

- Centraliza regras e configurações, como autenticação ou segurança.
- Reduz duplicação de código em rotas.
- Torna o sistema modular e escalável.

### Como configurar um Middleware no FastAPI

Middlewares no FastAPI são configurados com o decorador _@app.middleware_.

```python
from fastapi import FastAPI, Request
from starlette.responses import Response

app = FastAPI()

@app.middleware("http")
async def add_custom_header(request: Request, call_next):
    # Processa a requisição antes de enviar para as rotas
    response = await call_next(request)
    # Modifica a resposta antes de enviá-la ao cliente
    response.headers["X-Custom-Header"] = "FastAPI Middleware"
    return response

@app.get("/")
async def root():
    return {"message": "Bem-vindo ao middleware básico!"}
```



## Rate Limiting 



Rate Limiting é crucial para proteger APIs contra abuso, controlar a carga do sistema e evitar excessos de requisições. Para o FastAPI, há várias formas de implementar Rate Limiting.


### Por que usar Rate Limiting?

- Prevenir ataques de DDoS ou brute force.
- Melhorar o desempenho geral.
- Garantir que recursos sejam acessíveis a todos os usuários de forma equilibrada.

```bash
pip install slowapi
```


```python
from fastapi import FastAPI, Request, HTTPException
from slowapi import Limiter
from slowapi.util import get_remote_address
from slowapi.errors import RateLimitExceeded
from starlette.responses import JSONResponse

# Criação do app
app = FastAPI()

# Configurando o limiter
limiter = Limiter(key_func=get_remote_address)

# Integrando o Limiter ao FastAPI
@app.middleware("http")
async def rate_limit_middleware(request: Request, call_next):
    try:
        response = await limiter.limit("5/minute")(call_next)(request)
        return response
    except RateLimitExceeded as e:
        return JSONResponse(
            status_code=429,
            content={"detail": "Too Many Requests"},
        )

# Rotas
@app.get("/")
@limiter.limit("10/minute")
async def root():
    return {"message": "Bem-vindo! Você pode acessar esta rota 10 vezes por minuto."}

@app.get("/custom")
@limiter.limit("3/minute")
async def custom_limit():
    return {"message": "Esta rota tem um limite personalizado de 3 requisições por minuto."}
```



### Personalização de Rate Limiting

- Definir diferentes limites por rota: Use decoradores como _@limiter.limit("50/hour")_ ou _@limiter.limit("1/second")_.

- Exceções específicas: Permita que algumas rotas sejam ilimitadas:
    ```python
    @limiter.exempt
    @app.get("/health")
    async def health_check():
        return {"status": "OK"}
    ```




## Gerenciamento de Permissões (RBAC)



_Role-Based Access Control_ é o controle baseado em papéis (roles), permitindo ou negando acesso às rotas de acordo com as permissões de cada papel.

```python
from fastapi import FastAPI, Request, HTTPException

app = FastAPI()

@app.middleware("http")
async def rbac_middleware(request: Request, call_next):
    role = request.headers.get("X-Role")
    if not role:
        raise HTTPException(status_code=403, detail="Role não fornecido")
    
    # Verifique permissões baseadas no papel
    if request.url.path == "/admin" and role != "admin":
        raise HTTPException(status_code=403, detail="Acesso negado para este papel")
    
    response = await call_next(request)
    return response

@app.get("/")
async def root():
    return {"message": "Rota pública"}

@app.get("/admin")
async def admin_route():
    return {"message": "Bem-vindo, Admin!"}
```


## Rate Limiting com Middleware

```python
from fastapi import FastAPI, Request, HTTPException
from slowapi import Limiter
from slowapi.util import get_remote_address
from slowapi.errors import RateLimitExceeded
from starlette.responses import JSONResponse

app = FastAPI()

# Configurando o Limiter
limiter = Limiter(key_func=get_remote_address)

@app.middleware("http")
async def combined_middleware(request: Request, call_next):
    # RBAC
    role = request.headers.get("X-Role")
    if not role:
        raise HTTPException(status_code=403, detail="Role não fornecido")
    if request.url.path == "/admin" and role != "admin":
        raise HTTPException(status_code=403, detail="Acesso negado")

    # Rate Limiting
    try:
        response = await limiter.limit("5/minute")(call_next)(request)
        return response
    except RateLimitExceeded:
        return JSONResponse(
            status_code=429,
            content={"detail": "Too Many Requests"}
        )

@app.get("/")
async def public_route():
    return {"message": "Rota pública"}

@app.get("/admin")
@limiter.limit("2/minute")
async def admin_route():
    return {"message": "Bem-vindo, Admin!"}
```

## Nova estrutura proposta:

├── main.py
├── routers/
│   ├── tasks.py
│   ├── users.py
├── models/
│   ├── task.py
│   ├── user.py
├── schemas/
│   ├── tasks.py
│   ├── users.py
├── db/
│   ├── database.py
├── auth/
│   ├── dependencies.py
│   ├── jwt_handler.py
├── config/
│   ├── limiter.py
├── middlewares/
│   ├── auth.py
│   ├── rate_limit.py
│   ├── rbac.py
├── .venv/
├── requirements.txt
├── aula6.ipynb


Pra fazer tudo isso, também precisamos atualizar o banco de dados, o schema e o model do nosso projeto.